In [1]:
# from scipy.optimize import linprog
from scipy_solver.linprog import linprog
from tqdm import tqdm

In [2]:
import numpy as np
import scipy.sparse


def generate_setcover(nrows, ncols, density, max_coef=100):
    """
    Generates a setcover instance with specified characteristics, and writes
    it to a file in the LP format.

    Approach described in:
    E.Balas and A.Ho, Set covering algorithms using cutting planes, heuristics,
    and subgradient optimization: A computational study, Mathematical
    Programming, 12 (1980), 37-60.

    Parameters
    ----------
    nrows : int
        Desired number of rows
    ncols : int
        Desired number of columns
    density: float between 0 (excluded) and 1 (included)
        Desired density of the constraint matrix
    filename: str
        File to which the LP will be written
    rng: numpy.random.RandomState
        Random number generator
    max_coef: int
        Maximum objective coefficient (>=1)
    """
    nnzrs = int(nrows * ncols * density)

    assert nnzrs >= nrows  # at least 1 col per row
    assert nnzrs >= 2 * ncols  # at leats 2 rows per col

    # compute number of rows per column
    indices = np.random.choice(ncols, size=nnzrs)  # random column indexes
    indices[:2 * ncols] = np.repeat(np.arange(ncols), 2)  # force at leats 2 rows per col
    _, col_nrows = np.unique(indices, return_counts=True)

    # for each column, sample random rows
    indices[:nrows] = np.random.permutation(nrows) # force at least 1 column per row
    i = 0
    indptr = [0]
    for n in col_nrows:

        # empty column, fill with random rows
        if i >= nrows:
            indices[i:i+n] = np.random.choice(nrows, size=n, replace=False)

        # partially filled column, complete with random rows among remaining ones
        elif i + n > nrows:
            remaining_rows = np.setdiff1d(np.arange(nrows), indices[i:nrows], assume_unique=True)
            indices[nrows:i+n] = np.random.choice(remaining_rows, size=i+n-nrows, replace=False)

        i += n
        indptr.append(i)

    # objective coefficients
    c = np.random.rand(ncols) * max_coef

    # sparce CSC to sparse CSR matrix
    A = scipy.sparse.csc_matrix(
            (np.ones(len(indices), dtype=float), indices, indptr),
            shape=(nrows, ncols)).toarray()
    
    A = -A
    b = np.ones(nrows, dtype=np.float32) * -1
    A = np.concatenate([A, np.eye(nrows, dtype=np.float32)], axis=1)
    c = np.concatenate([c, np.zeros(nrows, dtype=np.float32)], axis=0)
    
    return A, b, c

In [3]:
import gzip
import pickle
import torch
from scipy.linalg import LinAlgWarning
from scipy.optimize._optimize import OptimizeWarning

In [ ]:
import warnings
warnings.filterwarnings("error")

ips = []
max_iter = 120000
num = 100000

pkg_idx = 0

success_cnt = 0
fail_cnt = 0

for i in tqdm(range(max_iter)):
    A, b, c = generate_setcover(nrows=np.random.randint(30, 50), ncols=np.random.randint(50, 70), density=0.3, max_coef=1.)
    
    try:
        res = linprog(c, 
                A_ub=None, 
                b_ub=None, 
                A_eq=A, b_eq=b, bounds=None, method='interior-point')
    except (LinAlgWarning, OptimizeWarning):
        fail_cnt += 1
        continue
    else:
        if res.success:
            ips.append((torch.from_numpy(A).to(torch.float), torch.from_numpy(b).to(torch.float), torch.from_numpy(c).to(torch.float)))
            success_cnt += 1

    if len(ips) >= 1000 or success_cnt == num:
        with gzip.open(f'small_instances/raw/instance_{pkg_idx}.pkl.gz', "wb") as file:
            pickle.dump(ips, file)
            pkg_idx += 1
        ips = []

    if success_cnt >= num:
        break

In [ ]:
fail_cnt

In [ ]:
warnings.resetwarnings()

In [ ]:
cnt = 0 
for i in range(90, 100):
    with gzip.open(f'small_instances/raw/instance_{i}.pkl.gz', "rb") as file:
        ip_pkgs = pickle.load(file)

    for ip_idx in range(len(ip_pkgs)):
        cnt += 1
        (A, b, c) = ip_pkgs[ip_idx]
        c = c / c.max()  # does not change the result
        
        sol = linprog(c.numpy(),
                      A_ub=None,
                      b_ub=None,
                      A_eq=A.numpy(), b_eq=b.numpy(), bounds=None,
                      method='interior-point')